In [3]:
#import SparkSession
from pyspark.sql import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#create spar session object
spark=SparkSession.builder.appName('data_processing').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Carga de datos del CSV Covid-19 desde Bucket S3
df=spark.read.csv('s3://jaromanpdatasets/datasets/spark/covid_data.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Columnas del dataframe
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado', 'fecha reporte web']

In [10]:
#Numero de columnas
len(df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16

In [11]:
#Numero de registros (En este caso seria el número total de casos registrados hasta el momento)
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6211

In [12]:
#Esquema de los datos
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Codigo DIVIPOLA: integer (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: timestamp (nullable = true)

In [13]:
#Informacion del dataset
df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+-------------------+------------------------+--------+------------------+----+-----------+---------+-------------------+--------------------+--------------------+--------------------+
|summary|        ID de caso|   Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención|              Edad|Sexo|       Tipo|   Estado|País de procedencia|                 FIS|     Fecha de muerte|    Fecha recuperado|
+-------+------------------+------------------+-------------------+------------------------+--------+------------------+----+-----------+---------+-------------------+--------------------+--------------------+--------------------+
|  count|              6211|              6211|               6211|                    6211|    6211|              6211|6211|       6211|     6211|               6211|                6211|                6211|                5952|
|   mean|            3106.0|30742.691515053935|               null|         

In [14]:
from pyspark.sql.types import StringType,DoubleType,IntegerType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
#Seleccio de columnas
df.select('Ciudad de ubicación','Sexo').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+
|Ciudad de ubicación|Sexo|
+-------------------+----+
|        Bogotá D.C.|   F|
|Guadalajara de Buga|   M|
|           Medellín|   F|
|           Medellín|   M|
|           Medellín|   M|
+-------------------+----+
only showing top 5 rows

In [20]:
from pyspark.sql.functions import datediff, to_date, lit, unix_timestamp

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#Creación de columna
df.withColumn("Dias desde que se descubrio el caso", 
              datediff(to_date(lit("2020-05-01")),
                       to_date(unix_timestamp('Fecha diagnostico', "yyyy/MM/dd").cast("timestamp")))).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+------------------------------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado| País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|dias que han pasado desde que se descubrio|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+------------------------------------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|

In [29]:
#Filtro Mujeres mayores de 40
df.filter(df['Sexo']=='F').filter(df['edad'] >40).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado| País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|         3|  2020-03-07 00:00:00|           5001|           Medellín|               Antioquia|Recuperado|  50|   F|  Importado|  Leve|              España|2020-02-29T00:00:...|          -   -|2020-03-09 00:00:00|2020-03-15T00:00:...|2

In [30]:
#Condición ciudad Armenia y Atención recuperado
df.filter((df['Ciudad de ubicación']=='Armenia')&(df['atención']=="Recuperado")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+--------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|  Estado| País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+--------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|        76|  2020-03-14 00:00:00|          63001|            Armenia|                 Quindio|Recuperado|  46|   F|  Importado|    Leve|              Italia|2020-03-13T00:00:...|          -   -|2020-03-17 00:00:00|2020-03-29T00:

In [31]:
#Agrupamiento por mayor cantidad de casos en ciudades 
df.groupBy('Ciudad de ubicación').count().orderBy('count',ascending=False).show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|Ciudad de ubicación|count|
+-------------------+-----+
|Bogotá D.C.        |2543 |
|Cali               |713  |
|Villavicencio      |344  |
|Medellín           |305  |
|Cartagena de Indias|246  |
+-------------------+-----+
only showing top 5 rows

In [32]:
# Agrupación por tipo de atención y promedio
df.groupBy('Atención').mean().show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------------+--------------------+------------------+
|Atención    |avg(ID de caso)   |avg(Codigo DIVIPOLA)|avg(Edad)         |
+------------+------------------+--------------------+------------------+
|Hospital UCI|2789.982905982906 |37169.666666666664  |56.666666666666664|
|Fallecido   |2409.248201438849 |33502.374100719426  |66.38129496402878 |
|Casa        |3746.7800246609127|30915.82540073983   |38.25647348951911 |
|Hospital    |4205.312865497076 |34303.26023391813   |52.73391812865497 |
|N/A         |4801.0            |11001.0             |20.0              |
+------------+------------------+--------------------+------------------+
only showing top 5 rows

In [36]:
from pyspark.sql.functions import udf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
#Cración de función
def danger_range(age, atencion):
    if atencion in ['Hospital UCI','Hospital']:
        if age > 60:
            return 'Peligro alto'
        else:
            return 'Peligro moderado'
    elif atencion == 'Casa':
        if age < 60:
            return 'Poco peligro'
        else:
            return 'Peligro moderado'
    elif atencion == "Recuperado":
        return 'Sin peligro'
    else:
        return 'Sin clasificación'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
#create udf using python function
age_udf=udf(danger_range,StringType())
#apply udf on dataframe
df.withColumn('danger_range',age_udf(df['Edad'], df['atención'])).show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención  |Edad|Sexo|Tipo       |Estado|País de procedencia      |FIS                    |Fecha de muerte|Fecha diagnostico  |Fecha recuperado       |fecha reporte web  |danger_range|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+------------+
|1         |2020-03-02 00:00:00  |11001          |Bogotá D.C.        |Bogotá D.C.             |Recuperado|19  |F   |Importado  |Leve  |Italia                   |20

In [43]:
#Guardando datos (csv)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
#Directorio actual
pwd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
#directorio objetivo
write_uri='s3://jaromanpdatasets/datasets/spark/df_csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
#Guardando dataframe como un solo archivo csv
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
#Guardado de datos en formato parket

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
#directorio objetivo
parquet_uri='s3://jaromanpdatasets/datasets/spark/df_parquet'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
#Salvado de datos en parquet
df.write.format('parquet').save(parquet_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…